In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [52]:
data = [
    [True, True, False, True, True],
    [False, True, False, False, True],
    [True, True, False, True, False],
    [True, True, True, False, True],
    [True, False, False, False, True],
]

import numpy as np

np.append(np.array(data).flatten(), True)

array([ True,  True, False,  True,  True, False,  True, False, False,
        True,  True,  True, False,  True, False,  True,  True,  True,
       False,  True,  True, False, False, False,  True,  True])

In [53]:
from agent import DQNAgent
import asyncio
import websockets
import json

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from agent import DQNAgent
import numpy as np




In [54]:
masked = torch.tensor(
    [
        [
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            -6.6584e-02,
        ]
    ]
)

print(masked)

masked.argmax()

tensor([[ 1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,
          1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,
          1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,
          1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,
          1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,
         -6.6584e-02]])


tensor(0)

In [55]:
a = torch.randn(1, 26)
print(a)
a.argmax().item()

tensor([[-0.1640,  1.0494, -0.4872, -0.5114, -1.6136,  0.6104, -0.8561,  0.7891,
         -0.7939, -0.5400, -1.6846,  1.2589, -1.8010,  1.9141,  1.0155, -0.7969,
         -0.4034, -0.3652,  1.3241,  0.4193, -0.7024,  1.8763,  1.0736,  1.1222,
          0.1502,  0.1395]])


13

In [56]:
batch_size = 32
channels = 1
board_height = 5
board_width = 5
tensor_cnn = torch.tensor([batch_size, channels, board_height, board_width])

In [57]:
a = torch.zeros([1,5,5])
print(a)

b = torch.ones([2, *a.shape])
b[0] = a
b

tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])


tensor([[[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]]])

In [124]:
def preprocess_state(board: list[str]):
    """
    Convert the board (list of strings, e.g. ["XX.O.", ...]) into a float tensor.
    For example:
    'X' -> 1.0  (Black)
    'O' -> -1.0 (White)
    '.' -> 0.0  (Empty)
    '#' -> 0.0  (Dead node, treat as non-playable)
    etc.
    ------------------------------
    """
    w = 5
    h = 5
    new_tensor = torch.zeros([1, 1, w, h], dtype=torch.float)
    for x in range(w):
        for y in range(h):
            ch = board[x][y]
            if ch == "X":
                new_tensor[0][0][x][y] = 1.0
            elif ch == "Y":
                new_tensor[0][0][x][y] = -1.0
            else:
                new_tensor[0][0][x][y] = 0.0

    return new_tensor  # shape [1, 1, 5, 5]


In [125]:
%%timeit
preprocess_state(
    [
        "#X...",
        "#Y...",
        "#....",
        "#....",
        "#....",
    ]
)

154 μs ± 1.87 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
def preprocess_state2(board_state_strs):
    """
    Convert the board (list of strings, e.g. ["XX.O.", ...]) into a float tensor of shape [1,1,w,h].
    'X' -> 1.0  (Black)
    'O' -> -1.0 (White)
    '.' -> 0.0  (Empty)
    '#' -> 0.0  (Dead node, treat as non-playable, will be masked out later anyways)
    """

    w, h = 5, 5
    board_2d = []
    for x in range(w):
        row = []
        for y in range(h):
            ch = board_state_strs[x][y]
            if ch == "X":
                row.append(1.0)
            elif ch == "Y":
                row.append(-1.0)
            else:
                row.append(0.0)
        board_2d.append(row)

    # Convert to Tensor shape [5, 5]
    board_tensor = torch.tensor(board_2d, dtype=torch.float)
    # Add batch and channel dims => shape [1, 1, 5, 5]
    board_tensor = board_tensor.unsqueeze(0).unsqueeze(0)

    return board_tensor

In [129]:
preprocess_state2(
    [
        "#X...",
        "#Y...",
        "#....",
        "#....",
        "#....",
    ]
)

[[0.0, 1.0, 0.0, 0.0, 0.0], [0.0, -1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0]]


tensor([[[[ 0.,  1.,  0.,  0.,  0.],
          [ 0., -1.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]]]])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

a = torch.tensor([1,2], device=device)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.